In [1]:
import os
import math
import folium
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon

from utils import *

In [10]:
d = {
    'id': [1, 2, 3, 4], 
    'longitude': [-73.589462, -73.590055, -73.590041, -73.589008],
    'latitude': [45.592012, 45.592189, 45.592181, 45.590781],
    'poly_id': [99, 99, 99, 99]
}
df = pd.DataFrame(data=d)

gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
gdf

,id,longitude,latitude,poly_id,geometry
0,1,-73.589462,45.592012,99,POINT (-73.58946 45.59201)
1,2,-73.590055,45.592189,99,POINT (-73.59006 45.59219)
2,3,-73.590041,45.592181,99,POINT (-73.59004 45.59218)
3,4,-73.589008,45.590781,99,POINT (-73.58901 45.59078)


In [11]:
from scipy.spatial import cKDTree
import numpy as np

# Create a KDTree from the geometry of the GeoDataFrame
tree = cKDTree(np.array(gdf.geometry.apply(lambda geom: [geom.x, geom.y])).tolist())

# Query the tree for the closest points to each point in the GeoDataFrame
distances, indices = tree.query(np.array(gdf.geometry.apply(lambda geom: [geom.x, geom.y])).tolist(), k=2)

# Get the closest point for each row
gdf['closest_point'] = gdf.geometry.iloc[indices[:, 1]].values


In [12]:
gdf

,id,longitude,latitude,poly_id,geometry,closest_point
0,1,-73.589462,45.592012,99,POINT (-73.58946 45.59201),POINT (-73.59004 45.59218)
1,2,-73.590055,45.592189,99,POINT (-73.59006 45.59219),POINT (-73.59004 45.59218)
2,3,-73.590041,45.592181,99,POINT (-73.59004 45.59218),POINT (-73.59006 45.59219)
3,4,-73.589008,45.590781,99,POINT (-73.58901 45.59078),POINT (-73.58946 45.59201)


In [13]:
m = folium.Map(location=[45.5414195, -73.6303031], tiles="cartodbpositron", zoom_start=12)

In [14]:
for _, row in gdf.iterrows():
    lat = row.latitude
    lon = row.longitude
    folium.Marker(location=[lat, lon], popup=row["id"]).add_to(m)

In [15]:
m

In [56]:
d = {
    'id': [1, 2, 3, 4, 5, 6, 7, 8], 
    'longitude': [-73.589462, -73.590055, -73.590041, -73.589008, -73.605997, -73.606028, -73.606822, -73.606832],
    'latitude': [45.592012, 45.592189, 45.592181, 45.590781, 45.590096, 45.589438, 45.589075, 45.589065],
    'poly_id': [99, 99, 99, 99, 100, 100, 100, 100]
}
df = pd.DataFrame(data=d)

gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
gdf

,id,longitude,latitude,poly_id,geometry
0,1,-73.589462,45.592012,99,POINT (-73.58946 45.59201)
1,2,-73.590055,45.592189,99,POINT (-73.59006 45.59219)
2,3,-73.590041,45.592181,99,POINT (-73.59004 45.59218)
3,4,-73.589008,45.590781,99,POINT (-73.58901 45.59078)
4,5,-73.605997,45.590096,100,POINT (-73.60600 45.59010)
5,6,-73.606028,45.589438,100,POINT (-73.60603 45.58944)
6,7,-73.606822,45.589075,100,POINT (-73.60682 45.58908)
7,8,-73.606832,45.589065,100,POINT (-73.60683 45.58906)


In [57]:
from scipy.spatial import cKDTree
import numpy as np

# Define a function to find the closest point for each group
def get_closest_points(group):
    # Create a KDTree from the geometry of the group
    tree = cKDTree(np.array(group.geometry.apply(lambda geom: [geom.x, geom.y])).tolist())
    
    # Query the tree for the closest points to each point in the group
    distances, indices = tree.query(np.array(group.geometry.apply(lambda geom: [geom.x, geom.y])).tolist(), k=2)

    # Get the closest point for each row in the group
    group['closest_point'] = group.geometry.iloc[indices[:, 1]].values
    
    return group

# Apply the function to each group defined by the 'poly_id' column
gdf = gdf.groupby('poly_id', group_keys=False).apply(get_closest_points).reset_index(drop=True)


In [58]:
gdf

,id,longitude,latitude,poly_id,geometry,closest_point
0,1,-73.589462,45.592012,99,POINT (-73.58946 45.59201),POINT (-73.59004 45.59218)
1,2,-73.590055,45.592189,99,POINT (-73.59006 45.59219),POINT (-73.59004 45.59218)
2,3,-73.590041,45.592181,99,POINT (-73.59004 45.59218),POINT (-73.59006 45.59219)
3,4,-73.589008,45.590781,99,POINT (-73.58901 45.59078),POINT (-73.58946 45.59201)
4,5,-73.605997,45.590096,100,POINT (-73.60600 45.59010),POINT (-73.60603 45.58944)
5,6,-73.606028,45.589438,100,POINT (-73.60603 45.58944),POINT (-73.60600 45.59010)
6,7,-73.606822,45.589075,100,POINT (-73.60682 45.58908),POINT (-73.60683 45.58906)
7,8,-73.606832,45.589065,100,POINT (-73.60683 45.58906),POINT (-73.60682 45.58908)


In [30]:
m2 = folium.Map(location=[45.5414195, -73.6303031], tiles="cartodbpositron", zoom_start=12)

In [31]:
for _, row in gdf.iterrows():
    lat = row.latitude
    lon = row.longitude
    folium.Marker(location=[lat, lon], popup=row["id"]).add_to(m2)

In [32]:
m2

In [38]:
result = result.geometry.set_crs('epsg:4326')

In [66]:
from geopy.distance import distance

def get_distance(row):
    point = row['geometry']
    closest_point = row['closest_point']
    return distance((point.y, point.x), (closest_point.y, closest_point.x)).meters

gdf.apply(get_distance, axis=1)


0     48.928656
1      1.408537
2      1.408537
3    141.329531
4     73.172291
5     73.172291
6      1.358022
7      1.358022
dtype: float64

In [65]:
def get_distance(row):
    return row.geometry.distance(row.closest_point)
    
gdf.apply(get_distance, axis=1)


0    0.000603
1    0.000016
2    0.000016
3    0.001312
4    0.000659
5    0.000659
6    0.000014
7    0.000014
dtype: float64